In [2]:
from collections import defaultdict
import time

start = time.time()
poss = defaultdict(set)
for a in range(7):
    for b in range(7):
        for c in range(7):
            for d in range(7):
                for e in range(7):
                    if not a < 4:
                        continue
                    if not c == d:
                        continue
                    if not a + b + c + d + e == 21:
                        continue
                    if [a,b,c,d,e].count(0) > 1:
                        continue
                    if [a,b,c,d,e].count(1) > 3:
                        continue
                    if [a,b,c,d,e].count(2) > 3:
                        continue
                    if [a,b,c,d,e].count(3) > 4:
                        continue
                    if [a,b,c,d,e].count(4) > 2:
                        continue    
                    if [a,b,c,d,e].count(5) > 8:
                        continue
                    if [a,b,c,d,e].count(6) > 5:
                        continue
                    poss['a'].add(a)
                    poss['b'].add(b)
                    poss['c'].add(c)
                    poss['d'].add(d)
                    poss['e'].add(e)

print(f"Brute force took {(time.time() - start)*1000:.2f}ms")
print(poss)

Brute force took 5.32ms
defaultdict(<class 'set'>, {'a': {0, 1, 2, 3}, 'b': {0, 1, 2, 3, 4, 5, 6}, 'c': {3, 4, 5, 6}, 'd': {3, 4, 5, 6}, 'e': {0, 1, 2, 3, 4, 5, 6}})


In [3]:
from ortools.sat.python import cp_model

start = time.time()
# Create the model
model = cp_model.CpModel()

# Create variables for each cell (0-6 pips)
xs = []
for i in range(10):
    xs.append(model.NewIntVar(0, 6, f"x_{i}"))

for region in range(3):
    for cell in range(2):
        model.add(xs[region*3 + cell] == xs[region*3 + cell + 1])

# sum regions 9
# Constraint 3: a + b + c + d + e == 21
model.Add(sum(xs) == 41-9)

# Domino constraints - count occurrences of each pip value
for pip_value in range(7):
    count_vars = []
    for cell in xs:
        # Create boolean variable: is this cell equal to pip_value?
        is_equal = model.NewBoolVar(f'cell_eq_{pip_value}')
        model.Add(cell == pip_value).OnlyEnforceIf(is_equal)
        model.Add(cell != pip_value).OnlyEnforceIf(is_equal.Not())
        count_vars.append(is_equal)
    
    # Sum of boolean variables = count of this pip value
    count = sum(count_vars)
    
    # Apply the count constraints
    if pip_value == 0:
        model.Add(count <= 2)
    elif pip_value == 1:
        model.Add(count <= 4)
    elif pip_value == 2:
        model.Add(count <= 4)
    elif pip_value == 3:
        model.Add(count <= 2)
    elif pip_value == 4:
        model.Add(count <= 0)
    elif pip_value == 5:
        model.Add(count <= 1)
    elif pip_value == 6:
        model.Add(count <= 3)

# Solve and collect all solutions
solver = cp_model.CpSolver()

# Dictionary to store possible values for each variable
from collections import defaultdict
poss = defaultdict(set)

class SolutionCollector(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables, poss_dict):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._variables = variables
        self._poss = poss_dict
        self._solution_count = 0
    
    def on_solution_callback(self):
        self._solution_count += 1
        for i in range(len(xs)):
            self._poss[i].add(self.Value(self._variables[i]))
    
    def solution_count(self):
        return self._solution_count

solution_collector = SolutionCollector(xs, poss)
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, solution_collector)

print(f"Status: {solver.StatusName(status)}")
print(f"Solutions found: {solution_collector.solution_count()}")
print(f"CP-SAT took {(time.time() - start)*1000:.2f}ms")
print(f"Possible values: {dict(poss)}")

Status: OPTIMAL
Solutions found: 6
CP-SAT took 29.67ms
Possible values: {0: {1, 2, 6}, 1: {1, 2, 6}, 2: {1, 2, 6}, 3: {1, 2, 6}, 4: {1, 2, 6}, 5: {1, 2, 6}, 6: {1, 2, 6}, 7: {1, 2, 6}, 8: {1, 2, 6}, 9: {5}}


In [4]:
from z3 import *
from collections import defaultdict

start = time.time()
# Create Z3 integer variables (0-6 pips)
a = Int('a')
b = Int('b')
c = Int('c')
d = Int('d')
e = Int('e')

cells = [a, b, c, d, e]

# Create solver
solver = Solver()

# Domain constraints (0-6 for each cell)
for cell in cells:
    solver.add(cell >= 0, cell <= 6)

# Constraint 1: a < 4
solver.add(a < 4)

# Constraint 2: c == d
solver.add(c == d)

# Constraint 3: a + b + c + d + e == 21
solver.add(a + b + c + d + e == 21)

# Helper function to count occurrences of a value
def count_occurrences(cells, value):
    return Sum([If(cell == value, 1, 0) for cell in cells])

# Domino constraints - limit occurrences of each pip value
solver.add(count_occurrences(cells, 0) <= 1)
solver.add(count_occurrences(cells, 1) <= 3)
solver.add(count_occurrences(cells, 2) <= 3)
solver.add(count_occurrences(cells, 3) <= 4)
solver.add(count_occurrences(cells, 4) <= 2)
solver.add(count_occurrences(cells, 5) <= 8)
solver.add(count_occurrences(cells, 6) <= 5)

# Collect all solutions
poss = defaultdict(set)
solution_count = 0

while solver.check() == sat:
    model = solver.model()
    solution_count += 1
    
    # Extract values
    a_val = model[a].as_long()
    b_val = model[b].as_long()
    c_val = model[c].as_long()
    d_val = model[d].as_long()
    e_val = model[e].as_long()
    
    # Add to possible values
    poss['a'].add(a_val)
    poss['b'].add(b_val)
    poss['c'].add(c_val)
    poss['d'].add(d_val)
    poss['e'].add(e_val)
    
    # Block this solution to find the next one
    solver.add(Or(a != a_val, b != b_val, c != c_val, d != d_val, e != e_val))

print(f"Solutions found: {solution_count}")
print(f"Z3 took {(time.time() - start)*1000:.2f}ms")
print(f"Possible values: {dict(poss)}")

Solutions found: 41
Z3 took 72.36ms
Possible values: {'a': {0, 1, 2, 3}, 'b': {0, 1, 2, 3, 4, 5, 6}, 'c': {3, 4, 5, 6}, 'd': {3, 4, 5, 6}, 'e': {0, 1, 2, 3, 4, 5, 6}}


In [5]:
mccw_input = """a[2:2] b
a
b
a b
a=2""".splitlines()
mccw_input = [x.strip() for x in mccw_input]

In [6]:
model = cp_model.CpModel()

x1 = model.new_bool_var('x1')
x2 = model.new_bool_var('x2')
x3 = model.new_bool_var('x3')
x4 = model.new_bool_var('x4')

model.add(x1 + x3 + 2*x4 == 2)
model.add(x2 + x3 == 1)

class SolutionCounter(cp_model.CpSolverSolutionCallback):
    def __init__(self):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.num_solutions = 0

    def on_solution_callback(self) -> None:
        self.num_solutions += 1
        

solver = cp_model.CpSolver()
solver.parameters.enumerate_all_solutions = True
counter = SolutionCounter()
status = solver.solve(model, counter)

print(f"Found {counter.num_solutions} solutions")

Found 2 solutions


In [7]:
# for primary items with no multiplicities: collect together x's for options that have that item, then generate a constraint saying sum of them == 1
# for primary items with multiplicity K and no slack: same as above, but sum == K
# for primary items with multiplicity and interval [L:U]: two constraints, sum >= L, sum <= U
# for secondary items not assigned a color: sum <= 1
# for secondary items assigned a color: make a boolean variable for each possible color of each colored secondary item, then: sum of them <= 1

In [15]:
type(x1 + 2*x2)

ortools.sat.python.cp_model_helper.SumArray